# The cars dataset - Cleaning

The cars dataset is a basic dataset of some cars and their mileage. Many versions of this dataset are available, we'll be using the one R has installed by default.

In this notebook we'll import the dataset and clean it up.

In [33]:
import pandas as pd

df = pd.read_csv("files/mpg.csv")  
    
df.head(10) 

,Unnamed: 0,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
5,6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact
6,7,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact
7,8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
8,9,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact
9,10,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact


It was imported fine, but the first column is all wrong: the CSV has an index, but that index was seen as an extra column (and another index was added).

In [34]:
df = pd.read_csv("files/mpg.csv", index_col=0) 
df.head(10)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact
7,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact
8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact
9,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact
10,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact


But what does all this mean? When looking at data there are a number of ways in which data can be dirty:

- Bad data (missing observations, dual observations, ...)
- Wrong structure (fields joined or spread out, ...)
- Dirty (wrong datatypes, string processing needed, ...)

The mpg-dataset has no bad data, and the structure is also fine. But there still are improvements.

By the way, what all the fields stand for is also well [explained](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.6/topics/mpg) in the R-documentation.

Let's start with miles per gallon (cty and hwy for city and highway respectively). In liters per 100km that would be:

Liters100km = 	(100 * 3.785411784)/(1.609344 * MPG)

In [35]:
df['clkm'] = [ (100 * 3.785411784)/(1.609344 * mpg) for mpg in df['cty']]
df['hwlkm'] = [ (100 * 3.785411784)/(1.609344 * mpg) for mpg in df['hwy']]

df.head(10)

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,clkm,hwlkm
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,13.067477,8.110848
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,11.200694,8.110848
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11.760729,7.587567
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,11.200694,7.840486
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,14.700911,9.046715
6,audi,a4,2.8,1999,6,manual(m5),f,18,26,p,compact,13.067477,9.046715
7,audi,a4,3.1,2008,6,auto(av),f,18,27,p,compact,13.067477,8.711651
8,audi,a4 quattro,1.8,1999,4,manual(m5),4,18,26,p,compact,13.067477,9.046715
9,audi,a4 quattro,1.8,1999,4,auto(l5),4,16,25,p,compact,14.700911,9.408583
10,audi,a4 quattro,2.0,2008,4,manual(m6),4,20,28,p,compact,11.760729,8.400521


The class of a car is actually a categorical value. This means it can only have a number of discrete values. Let's convert the class of the car to that type.

In [36]:
df["class"] = pd.Categorical(df['class'])

The class of a car is not an ordered type of category but some are. Examples would be health labels on food, year a student is in, ... Let's assume it is though.

In [37]:
df['class'].unique()

['compact', 'midsize', 'suv', '2seater', 'minivan', 'pickup', 'subcompact']
Categories (7, object): ['2seater', 'compact', 'midsize', 'minivan', 'pickup', 'subcompact', 'suv']

In [38]:
from pandas.api.types import CategoricalDtype

# categories-list copied en rearranged from unique values
# and added a category just for fun
cat_type = CategoricalDtype(categories=['three wheeled car','2seater',
        'subcompact', 'compact', 'midsize', 'minivan', 'suv', 'pickup'], ordered=True)

df["class"] = df['class'].astype(cat_type)

df.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,clkm,hwlkm
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,13.067477,8.110848
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,11.200694,8.110848
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11.760729,7.587567
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,11.200694,7.840486
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,14.700911,9.046715


And why are we doing this? Well, let's look at the result of a group by now...

In [39]:
df.groupby('class').describe()

displ                                                 year  \
                  count      mean       std  min  25%   50%   75%  max count   
class                                                                          
three wheeled car   0.0       NaN       NaN  NaN  NaN   NaN   NaN  NaN   0.0   
2seater             5.0  6.160000  0.531977  5.7  5.7  6.20  6.20  7.0   5.0   
subcompact         35.0  2.660000  1.102457  1.6  1.9  2.20  3.25  5.4  35.0   
compact            47.0  2.325532  0.452274  1.8  2.0  2.20  2.80  3.3  47.0   
midsize            41.0  2.921951  0.718510  1.8  2.4  2.80  3.50  5.3  41.0   
minivan            11.0  3.390909  0.452669  2.4  3.3  3.30  3.80  4.0  11.0   
suv                62.0  4.456452  1.065805  2.5  4.0  4.65  5.30  6.5  62.0   
pickup             33.0  4.418182  0.828574  2.7  3.9  4.70  4.70  5.9  33.0   

                                ...       clkm            hwlkm             \
                          mean  ...        75%        max count       mean   
class                           ...                                          
three wheeled car          NaN  ...        NaN        NaN   0.0        NaN   
2seater            2004.400000  ...  15.680972  15.680972   5.0   9.505868   
subcompact         2003.114286  ...  13.836152  16.801042  35.0   8.624866   
compact            2003.212766  ...  13.067477  15.680972  47.0   8.434945   
midsize            2003.609756  ...  13.067477  15.680972  41.0   8.668584   
minivan            2003.090909  ...  15.190942  21.383144  11.0  10.615221   
suv                2003.790323  ...  19.601215  26.134954  62.0  13.302514   
pickup             2003.636364  ...  21.383144  26.134954  33.0  14.207798   

                                                                         \
                        std        min        25%        50%        75%   
class                                                                     
three wheeled car       NaN        NaN        NaN        NaN        NaN   
2seater            0.509315   9.046715   9.046715   9.408583   9.800608   
subcompact         1.479653   5.345786   7.730652   9.046715   9.604595   
compact            0.958972   5.345786   8.110848   8.711651   9.046715   
midsize            0.662133   7.350456   8.110848   8.711651   9.046715   
minivan            1.170082   9.800608   9.800608  10.226721  10.691572   
suv                2.103434   8.711651  12.379715  13.451814  13.836152   
pickup             2.128522  10.691572  13.067477  13.836152  14.700911   

                              
                         max  
class                         
three wheeled car        NaN  
2seater            10.226721  
subcompact         11.760729  
compact            10.226721  
midsize            10.226721  
minivan            13.836152  
suv                19.601215  
pickup             19.601215  

[8 rows x 56 columns]

Some selections! Let's get all cars with an engine displacement of 3 or less.

In [40]:
df[ df.displ <= 3]

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class,clkm,hwlkm
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,13.067477,8.110848
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,11.200694,8.110848
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11.760729,7.587567
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,11.200694,7.840486
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,14.700911,9.046715
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,volkswagen,passat,1.8,1999,4,auto(l5),f,18,29,p,midsize,13.067477,8.110848
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,12.379715,8.400521
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,11.200694,8.110848
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,14.700911,9.046715


And from this, only show the manufacturer and the nr of cylinders.

In [12]:
df[ df.displ <= 3][['manufacturer', 'cyl']]

,manufacturer,cyl
1,audi,4
2,audi,4
3,audi,4
4,audi,4
5,audi,6
...,...,...
229,volkswagen,4
230,volkswagen,4
231,volkswagen,4
232,volkswagen,6


All cars having an odd number of cylinders or a displacement of exactly 2.8.

In [13]:
df[ (df.displ == 2.8) | (df.cyl % 2 == 1)][['manufacturer', 'cyl', 'displ']]

,manufacturer,cyl,displ
5,audi,6,2.8
6,audi,6,2.8
12,audi,6,2.8
13,audi,6,2.8
16,audi,6,2.8
212,volkswagen,6,2.8
218,volkswagen,5,2.5
219,volkswagen,5,2.5
220,volkswagen,6,2.8
221,volkswagen,6,2.8


Same as above, but sort by ascending number of cylinders.

In [15]:
df[ (df.displ == 2.8) | (df.cyl % 2 == 1)][['manufacturer', 'cyl', 'displ']].sort_values('cyl')

,manufacturer,cyl,displ
218,volkswagen,5,2.5
219,volkswagen,5,2.5
226,volkswagen,5,2.5
227,volkswagen,5,2.5
5,audi,6,2.8
6,audi,6,2.8
12,audi,6,2.8
13,audi,6,2.8
16,audi,6,2.8
212,volkswagen,6,2.8


What is the average displacement of all our cars? [Many options!](https://medium.com/analytics-vidhya/how-to-summarize-data-with-pandas-2c9edffafbaf)

info(): provides a concise summary of a dataframe.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 1 to 234
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   manufacturer  234 non-null    object  
 1   model         234 non-null    object  
 2   displ         234 non-null    float64 
 3   year          234 non-null    int64   
 4   cyl           234 non-null    int64   
 5   trans         234 non-null    object  
 6   drv           234 non-null    object  
 7   cty           234 non-null    int64   
 8   hwy           234 non-null    int64   
 9   fl            234 non-null    object  
 10  class         193 non-null    category
 11  clkm          234 non-null    float64 
 12  hwlkm         234 non-null    float64 
dtypes: category(1), float64(3), int64(4), object(5)
memory usage: 24.3+ KB


describe(): Generates descriptive statistics that will provide visibility of the dispersion and shape of a dataset’s distribution. It excludes NaN values. It can be used for dataframe or a specific series.

In [17]:
df.describe()

,displ,year,cyl,cty,hwy,clkm,hwlkm
count,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171,14.841315,10.733481
std,1.291959,4.509646,1.611534,4.255946,5.954643,3.757080,2.898756
min,1.600000,1999.000000,4.000000,9.000000,12.000000,6.720417,5.345786
25%,2.400000,1999.000000,4.000000,14.000000,18.000000,12.379715,8.711651
50%,3.300000,2003.500000,6.000000,17.000000,24.000000,13.836152,9.800608
75%,4.600000,2008.000000,8.000000,19.000000,27.000000,16.801042,13.067477
max,7.000000,2008.000000,8.000000,35.000000,44.000000,26.134954,19.601215


value_counts(): returns counts of unique values for the specified series. NaN values are excluded by default.

In [18]:
df.displ.value_counts()

2.0    21
2.5    20
4.7    17
4.0    15
1.8    14
2.4    13
4.6    11
2.8    10
3.3     9
3.8     8
2.7     8
5.4     8
5.7     8
3.0     8
5.3     6
2.2     6
3.1     6
1.6     5
5.2     5
3.5     5
3.4     4
4.2     4
3.7     3
3.9     3
1.9     3
5.9     2
3.6     2
5.0     2
6.2     2
6.5     1
7.0     1
6.1     1
4.4     1
5.6     1
6.0     1
Name: displ, dtype: int64

nunique(): Count distinct observations. Can be used for a dataframe or a series. By default, it exclude the NaN values.

In [19]:
df.nunique()

manufacturer    15
model           38
displ           35
year             2
cyl              4
trans           10
drv              3
cty             21
hwy             27
fl               5
class            6
clkm            21
hwlkm           27
dtype: int64

sum(): Return the sum of the values for the requested axis. You can use it for both dataframe and series.

Watch out! Avoid selecting categorical columns to avoid a deprecated-error.

In [22]:
df[['displ', 'cyl', 'hwy','model']].sum()

# the error:
# df[['displ','class']].sum()

displ                                                812.4
cyl                                                   1378
hwy                                                   5485
model    a4a4a4a4a4a4a4a4 quattroa4 quattroa4 quattroa4...
dtype: object

count(): Return number of non-NA/null observations.

In [23]:
df.count( numeric_only = True)

displ    234
year     234
cyl      234
cty      234
hwy      234
clkm     234
hwlkm    234
dtype: int64

Min and Max, Mean and Median:

* min(): Return the minimum value
* max(): Return the maximum value
* mean(): Return the mean of the values
* median(): Return the median of the values

These functions can be applied to both dataframe and series.

(Note the class! The max is the last value of our ordered class.)

In [24]:
df.max()

manufacturer           volkswagen
model           toyota tacoma 4wd
displ                         7.0
year                         2008
cyl                             8
trans                  manual(m6)
drv                             r
cty                            35
hwy                            44
fl                              r
class                      pickup
clkm                    26.134954
hwlkm                   19.601215
dtype: object

agg(): apply more than one aggregation operations to the same dataset over the specified axis.

In [25]:
df[['displ', 'cyl', 'hwy','model']].agg(['count','min','max'])

,displ,cyl,hwy,model
count,234.0,234,234,234
min,1.6,4,12,4runner 4wd
max,7.0,8,44,toyota tacoma 4wd


groupby(): allows you to group data (by applying aggregate functions like sum, max, min…) with the same values into summary rows.

In [26]:
df.groupby('class').cyl.mean()

class
three wheeled car         NaN
2seater              8.000000
subcompact           5.028571
compact              4.595745
minivan              5.818182
suv                  6.967742
pickup               7.030303
Name: cyl, dtype: float64

When we look at the transmission of a car, it surely looks like there is more than one value in every cell. We'll use the pandas-split method to split this up.

In [27]:
df[["trans"]]

,trans
1,auto(l5)
2,manual(m5)
3,manual(m6)
4,auto(av)
5,auto(l5)
...,...
230,auto(s6)
231,manual(m6)
232,auto(l5)
233,manual(m5)


In [28]:
df["trans"].str.split('(')

1        [auto, l5)]
2      [manual, m5)]
3      [manual, m6)]
4        [auto, av)]
5        [auto, l5)]
           ...      
230      [auto, s6)]
231    [manual, m6)]
232      [auto, l5)]
233    [manual, m5)]
234      [auto, s6)]
Name: trans, Length: 234, dtype: object

So now we have a list. But what if we want a dataframe?

In [29]:
df["trans"].str.split('(', expand=True)

,0,1
1,auto,l5)
2,manual,m5)
3,manual,m6)
4,auto,av)
5,auto,l5)
...,...,...
230,auto,s6)
231,manual,m6)
232,auto,l5)
233,manual,m5)


Good! How could we get rid of the final ")"? To do that we need to store the output as a dataframe and apply a lambda function...

In [30]:
splitted = df["trans"].str.split('(', expand=True)
splitted[1] = splitted.apply(lambda row : row[1].replace(')',''), axis=1)
splitted

,0,1
1,auto,l5
2,manual,m5
3,manual,m6
4,auto,av
5,auto,l5
...,...,...
230,auto,s6
231,manual,m6
232,auto,l5
233,manual,m5
